# Binary Search 
Preprocessed file with unix command to 1). remove header 2). remove other chromosomes 3). Make sure that everything is protein coding and a transcript 

code: grep -v "#" BDGP6.Ensembl.81.gtf |grep -w "3R"|grep -w "protein_coding"|grep -w "gene" >preprocessed_protein_gene.gtf

In [1]:
import pandas as pd

In [84]:
#loading the data
gene_df=pd.read_csv("preprocessed_protein_gene.gtf",sep="\t", names=["chr","db","type","start","end","score","strand","frame","attribute"])
gene_df

,chr,db,type,start,end,score,strand,frame,attribute
0,3R,FlyBase,gene,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; gene_..."
1,3R,FlyBase,gene,2554124,3263573,.,-,.,"gene_id ""FBgn0267430""; gene_version ""1""; gene_..."
2,3R,FlyBase,gene,3322810,3354486,.,+,.,"gene_id ""FBgn0086917""; gene_version ""1""; gene_..."
3,3R,FlyBase,gene,3461351,3587054,.,-,.,"gene_id ""FBgn0010247""; gene_version ""1""; gene_..."
4,3R,FlyBase,gene,3623143,3627690,.,+,.,"gene_id ""FBgn0086378""; gene_version ""1""; gene_..."
...,...,...,...,...,...,...,...,...,...
3401,3R,FlyBase,gene,31978121,31990909,.,+,.,"gene_id ""FBgn0039886""; gene_version ""1""; gene_..."
3402,3R,FlyBase,gene,32038330,32039753,.,+,.,"gene_id ""FBgn0039887""; gene_version ""1""; gene_..."
3403,3R,FlyBase,gene,32045314,32051993,.,-,.,"gene_id ""FBgn0040206""; gene_version ""1""; gene_..."
3404,3R,FlyBase,gene,32052060,32054974,.,+,.,"gene_id ""FBgn0002780""; gene_version ""1""; gene_..."


In [85]:
#sanity check using panda's built-in search
gene_df['start'].searchsorted(21378950)


2104

In [99]:

search_param=21378950
def binary_search(database,search_param,extra_info=False):
    #upperbound index
    high=database.shape[0]
    #lower
    low=0
    count=0
    while low<high:
        middle=(high+low)//2
        start=int(database.iloc[middle]["start"])
        end=int(database.iloc[middle]["end"])
        if end<search_param:
            low=middle+1
        elif start>search_param:
            high=middle
        elif search_param in [start,end]:
            print(middle)
        count+=1
    gene_name=database.iloc[middle]["attribute"].split()[1][1:-2]
    if extra_info:
        print(middle,database.iloc[middle])
    return "The closest gene is {}. Our search method converged to this answer using binary search in {} steps".format(gene_name,count)

In [136]:
def recursive_binary(database,search_param,low,high,count,extra_info=False):
    middle=(high+low)//2
    if high<=low:
        gene_name=database.iloc[middle]["attribute"].split()[1][1:-2]
        if extra_info:
            print(middle,database.iloc[middle])
        return "The closest gene is {}. Our search method converged to this answer using binary search in {} steps".format(gene_name,count)
        
    
    start=int(database.iloc[middle]["start"])
    end=int(database.iloc[middle]["end"])
    if end<search_param:
        low=middle+1
        new_count=count+1
        return recursive_binary(database,search_param, low,high,count=new_count,extra_info=extra_info)
    elif start>search_param:
        high=middle
        new_count=count+1
        return recursive_binary(database,search_param,low,high,count=new_count,extra_info=extra_info)
    elif search_param in [start,end]:
        gene_name=database.iloc[middle]["attribute"].split()[1][1:-2]
        if extra_info:
            print(middle,database.iloc[middle])
        return "The closest gene is {}. Our search method converged to this answer using binary search in {} steps".format(gene_name,count)
        
    
    
    

In [137]:
print(recursive_binary(gene_df,search_param,low=0,high=gene_df.shape[0],count=0,extra_info=True))

2104 chr                                                         3R
db                                                     FlyBase
type                                                      gene
start                                                 21378977
end                                                   21381970
score                                                        .
strand                                                       +
frame                                                        .
attribute    gene_id "FBgn0004110"; gene_version "1"; gene_...
Name: 2104, dtype: object
The closest gene is FBgn0004110. Our search method converged to this answer using binary search in 12 steps


# Question 2 
Plan of attack. Start with closest gene in the middle. Have a left and right gene on each side. Check which is closer and print. Advance index on that side. Repeat until have 20 gene. Note, code only works if the set is orderted

In [88]:
left=middle-1
right=middle+1
count=0
closest_genes=[]
closest_genes.append((gene_name,min(abs(search_param-gene_df.iloc[middle]["start"]),abs(search_param-gene_df.iloc[middle]["end"]))))

while count<19:
    count+=1
    right_start=gene_df.iloc[right]["start"]
    left_end=gene_df.iloc[left]["end"]
    if abs(search_param-right_start)<abs(search_param-left_end):
        closest_genes.append(((gene_df.iloc[right]["attribute"].split()[1][1:-2]),abs(search_param-right_start),"right"))
        right+=1
    else:
        closest_genes.append(((gene_df.iloc[left]["attribute"].split()[1][1:-2]),abs(search_param-left_end),"left"))
        left-=1
print(closest_genes)

[('FBgn0004110', 27), ('FBgn0267652', 5375, 'left'), ('FBgn0267650', 5375, 'left'), ('FBgn0267651', 5375, 'left'), ('FBgn0261844', 9620, 'left'), ('FBgn0004862', 9930, 'right'), ('FBgn0261841', 10671, 'left'), ('FBgn0261840', 9620, 'left'), ('FBgn0261843', 10671, 'left'), ('FBgn0261839', 10972, 'left'), ('FBgn0261845', 9143, 'left'), ('FBgn0261842', 10671, 'left'), ('FBgn0266171', 13660, 'left'), ('FBgn0266170', 13660, 'left'), ('FBgn0266172', 13462, 'left'), ('FBgn0261837', 16819, 'left'), ('FBgn0261838', 16819, 'left'), ('FBgn0038886', 18831, 'right'), ('FBgn0266176', 19182, 'left'), ('FBgn0266175', 19182, 'left')]


# Question 3

In [106]:
all_genes=pd.read_csv("preprocessed_gene.gtf",sep="\t", names=["chr","db","type","start","end","score","strand","frame","attribute"])
all_genes

,chr,db,type,start,end,score,strand,frame,attribute
0,3R,FlyBase,gene,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; gene_..."
1,3R,FlyBase,gene,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; gene_..."
2,3R,FlyBase,gene,1031171,1031354,.,+,.,"gene_id ""FBgn0039987""; gene_version ""1""; gene_..."
3,3R,FlyBase,gene,2156916,2157206,.,-,.,"gene_id ""FBgn0058182""; gene_version ""1""; gene_..."
4,3R,FlyBase,gene,2554124,3263573,.,-,.,"gene_id ""FBgn0267430""; gene_version ""1""; gene_..."
...,...,...,...,...,...,...,...,...,...
4119,3R,FlyBase,gene,32015941,32026928,.,+,.,"gene_id ""FBgn0267117""; gene_version ""1""; gene_..."
4120,3R,FlyBase,gene,32038330,32039753,.,+,.,"gene_id ""FBgn0039887""; gene_version ""1""; gene_..."
4121,3R,FlyBase,gene,32045314,32051993,.,-,.,"gene_id ""FBgn0040206""; gene_version ""1""; gene_..."
4122,3R,FlyBase,gene,32052060,32054974,.,+,.,"gene_id ""FBgn0002780""; gene_version ""1""; gene_..."


In [107]:
print(binary_search(all_genes,21378950,extra_info=True))


2545 chr                                                         3R
db                                                     FlyBase
type                                                      gene
start                                                 21378977
end                                                   21381970
score                                                        .
strand                                                       +
frame                                                        .
attribute    gene_id "FBgn0004110"; gene_version "1"; gene_...
Name: 2545, dtype: object
The closest gene is FBgn0004110. Our search method converged to this answer using binary search in 12 steps
